In [7]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.5 MB/s eta 0:00:000:00:010:00:01:01
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.wh

In [50]:
train_file ='train/in.tsv.xz'
test_file = 'test-A/in.tsv.xz'
out_file = 'test-A/out.tsv'

In [10]:
from itertools import islice
import regex as re
import sys
from torchtext.vocab import build_vocab_from_iterator
import lzma
import pickle
import re
import torch
from torch import nn
from torch.utils.data import IterableDataset
import itertools
from torch.utils.data import DataLoader
import gc

In [51]:
embed_size = 300
device = 'cuda'
vocab_size = 25000
batch_s = 3200
learning_rate = 0.0001
epochs = 4
k = 20 #top k words
wildcard_minweight = 0.001

In [12]:
###preprocessing
def preprocess(line):
    line = get_rid_of_header(line)
    line = replace_endline(line)
    return line

def get_rid_of_header(line):
    line = line.split('\t')[6:]
    return "".join(line)
    
def replace_endline(line):
    line = line.replace("\\n", " ")
    return line


def get_last_word(text):
    """Return the last word of a string."""
    last_word = ""
    for i in range(len(text)-1, -1, -1):
        if text[i] == ' ':
            return last_word[::-1].rstrip()
        else:
            last_word += text[i]
    return last_word[::-1].rstrip()

def get_first_word(text):
    """Return the first word of a string."""
    word = ""
    for i in range(len(text)-1):
        if text[i] == ' ':
            return word
        else:
            word += text[i]
    return word


def get_words_from_line(line):
    line = line.rstrip()
    yield '<s>'
    line = preprocess(line)
    for t in line.split(' '):
        yield t
    yield '</s>'


def get_word_lines_from_file(file_name):
    n = 0
    with lzma.open(file_name, 'r') as fh:
        for line in fh:
            n+=1
            if n%1000==0:
                print(n)
            yield get_words_from_line(line.decode('utf-8'))

In [5]:
vocab = build_vocab_from_iterator(
    get_word_lines_from_file(train_file),
    max_tokens = vocab_size,
    specials = ['<unk>'])

with open('filename.pickle', 'wb') as handle:
    pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [5]:
vocab.lookup_tokens([0, 1, 2, 10, 2000])

NameError: name 'vocab' is not defined

#### Definicja sieci



Naszą prostą sieć neuronową zaimplementujemy używając frameworku PyTorch.



In [13]:
class SimpleBigramNeuralLanguageModel(nn.Module):
  def __init__(self, vocabulary_size, embedding_size):
      super(SimpleBigramNeuralLanguageModel, self).__init__()
      self.model = nn.Sequential(
          nn.Embedding(vocabulary_size, embedding_size),
          nn.Linear(embedding_size, vocabulary_size),
          nn.Softmax()
      )
    
  def forward(self, x):
      return self.model(x)

with open('filename.pickle','rb') as handle:
    vocab = pickle.load(handle)

vocab.set_default_index(vocab['<unk>'])

In [6]:
help(vocab)

Help on Vocab in module torchtext.vocab.vocab object:

class Vocab(torch.nn.modules.module.Module)
 |  Vocab(vocab) -> None
 |  
 |  Base class for all neural network modules.
 |  
 |  Your models should also subclass this class.
 |  
 |  Modules can also contain other Modules, allowing to nest them in
 |  a tree structure. You can assign the submodules as regular attributes::
 |  
 |      import torch.nn as nn
 |      import torch.nn.functional as F
 |  
 |      class Model(nn.Module):
 |          def __init__(self):
 |              super().__init__()
 |              self.conv1 = nn.Conv2d(1, 20, 5)
 |              self.conv2 = nn.Conv2d(20, 20, 5)
 |  
 |          def forward(self, x):
 |              x = F.relu(self.conv1(x))
 |              return F.relu(self.conv2(x))
 |  
 |  Submodules assigned in this way will be registered, and will have their
 |  parameters converted too when you call :meth:`to`, etc.
 |  
 |  .. note::
 |      As per the example above, an ``__init__()`` call

In [14]:
def look_ahead_iterator(gen):
    prev = None
    for item in gen:
        if prev is not None:
            yield (prev, item)
        prev = item

class Bigrams(IterableDataset):
  def __init__(self, text_file, vocabulary_size):
      self.vocab = build_vocab_from_iterator(
         get_word_lines_from_file(text_file),
         max_tokens = vocabulary_size,
         specials = ['<unk>'])
      self.vocab.set_default_index(self.vocab['<unk>'])
      self.vocabulary_size = vocabulary_size
      self.text_file = text_file

  def __iter__(self):
     return look_ahead_iterator(
         (self.vocab[t] for t in itertools.chain.from_iterable(get_word_lines_from_file(self.text_file))))

train_dataset = Bigrams(train_file, vocab_size)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [15]:
print(train_dataset)

In [17]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 699613 KiB |   1903 MiB |   3735 MiB |   3052 MiB |\n|       from large pool | 699414 KiB |   1903 MiB |   3734 MiB |   3051 MiB |\n|       from small pool |    199 KiB |      1 MiB |      1 MiB |      1 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         | 699613 KiB |   1903 MiB |   3735 MiB |   3052 MiB |\n|       from large pool | 699414 KiB |   1903 MiB |

In [10]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

In [12]:
device = 'cuda'
model = SimpleBigramNeuralLanguageModel(vocab_size, embed_size).to(device)

In [13]:
data = DataLoader(train_dataset, batch_size=batch_s)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.NLLLoss()
torch.cuda.empty_cache()
gc.collect()

model.load_state_dict(torch.load('model-bigram_final.bin'))
for i in range(1, epochs+1):
    print('epoch: =', i)
    model.train()
    step = 0
    for x, y in data: # prev, predicting, following words
       x = x.to(device)
       y = y.to(device)
       optimizer.zero_grad()
       ypredicted = model(x) #previous, following word
       loss = criterion(torch.log(ypredicted), y)
       if step % 100 == 0:
          print(step, loss)
       step += 1
       loss.backward()
       optimizer.step()
    torch.save(model.state_dict(), f'model-bigram_2nd-run{i}.bin')    
torch.save(model.state_dict(), f'model-bigram_final.bin')    

epoch: = 1


/home/gedin/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 tensor(5.9599, device='cuda:0', grad_fn=<NllLossBackward0>)
1000
100 tensor(6.1015, device='cuda:0', grad_fn=<NllLossBackward0>)
200 tensor(5.9708, device='cuda:0', grad_fn=<NllLossBackward0>)
2000
300 tensor(6.2176, device='cuda:0', grad_fn=<NllLossBackward0>)
3000
400 tensor(5.9401, device='cuda:0', grad_fn=<NllLossBackward0>)
4000
500 tensor(6.2084, device='cuda:0', grad_fn=<NllLossBackward0>)
5000
600 tensor(5.9736, device='cuda:0', grad_fn=<NllLossBackward0>)
6000
700 tensor(6.1423, device='cuda:0', grad_fn=<NllLossBackward0>)
7000
800 tensor(5.7344, device='cuda:0', grad_fn=<NllLossBackward0>)
8000
900 tensor(6.0950, device='cuda:0', grad_fn=<NllLossBackward0>)
9000
1000 tensor(5.8473, device='cuda:0', grad_fn=<NllLossBackward0>)
10000
1100 tensor(6.0612, device='cuda:0', grad_fn=<NllLossBackward0>)
11000
1200 tensor(6.1509, device='cuda:0', grad_fn=<NllLossBackward0>)
12000
1300 tensor(6.0760, device='cuda:0', grad_fn=<NllLossBackward0>)
13000
1400 tensor(6.2047, device='cuda:

114000
11600 tensor(5.6308, device='cuda:0', grad_fn=<NllLossBackward0>)
115000
11700 tensor(5.5680, device='cuda:0', grad_fn=<NllLossBackward0>)
116000
11800 tensor(5.7604, device='cuda:0', grad_fn=<NllLossBackward0>)
117000
11900 tensor(5.5792, device='cuda:0', grad_fn=<NllLossBackward0>)
118000
12000 tensor(5.7329, device='cuda:0', grad_fn=<NllLossBackward0>)
119000
12100 tensor(5.7726, device='cuda:0', grad_fn=<NllLossBackward0>)
120000
12200 tensor(5.7151, device='cuda:0', grad_fn=<NllLossBackward0>)
121000
12300 tensor(5.8561, device='cuda:0', grad_fn=<NllLossBackward0>)
122000
12400 tensor(5.6791, device='cuda:0', grad_fn=<NllLossBackward0>)
123000
12500 tensor(5.5574, device='cuda:0', grad_fn=<NllLossBackward0>)
124000
12600 tensor(5.6817, device='cuda:0', grad_fn=<NllLossBackward0>)
125000
12700 tensor(5.5375, device='cuda:0', grad_fn=<NllLossBackward0>)
126000
12800 tensor(5.7270, device='cuda:0', grad_fn=<NllLossBackward0>)
127000
12900 tensor(5.6252, device='cuda:0', grad_f

226000
22900 tensor(5.6646, device='cuda:0', grad_fn=<NllLossBackward0>)
227000
23000 tensor(5.5949, device='cuda:0', grad_fn=<NllLossBackward0>)
228000
23100 tensor(5.5284, device='cuda:0', grad_fn=<NllLossBackward0>)
229000
23200 tensor(5.5617, device='cuda:0', grad_fn=<NllLossBackward0>)
230000
23300 tensor(5.6426, device='cuda:0', grad_fn=<NllLossBackward0>)
231000
23400 tensor(5.7283, device='cuda:0', grad_fn=<NllLossBackward0>)
232000
23500 tensor(5.4558, device='cuda:0', grad_fn=<NllLossBackward0>)
233000
23600 tensor(5.4600, device='cuda:0', grad_fn=<NllLossBackward0>)
234000
23700 tensor(5.4961, device='cuda:0', grad_fn=<NllLossBackward0>)
235000
23800 tensor(5.3373, device='cuda:0', grad_fn=<NllLossBackward0>)
236000
23900 tensor(5.4470, device='cuda:0', grad_fn=<NllLossBackward0>)
237000
24000 tensor(5.4346, device='cuda:0', grad_fn=<NllLossBackward0>)
238000
24100 tensor(5.5112, device='cuda:0', grad_fn=<NllLossBackward0>)
239000
24200 tensor(5.6918, device='cuda:0', grad_f

...

372000
37600 tensor(5.3282, device='cuda:0', grad_fn=<NllLossBackward0>)
373000
37700 tensor(5.5061, device='cuda:0', grad_fn=<NllLossBackward0>)
374000
37800 tensor(5.3408, device='cuda:0', grad_fn=<NllLossBackward0>)
375000
37900 tensor(5.3203, device='cuda:0', grad_fn=<NllLossBackward0>)
376000
38000 tensor(5.3996, device='cuda:0', grad_fn=<NllLossBackward0>)
377000
38100 tensor(5.4133, device='cuda:0', grad_fn=<NllLossBackward0>)
378000
38200 tensor(5.4262, device='cuda:0', grad_fn=<NllLossBackward0>)
379000
38300 tensor(5.3305, device='cuda:0', grad_fn=<NllLossBackward0>)
380000
38400 tensor(5.3983, device='cuda:0', grad_fn=<NllLossBackward0>)
381000
38500 tensor(5.4246, device='cuda:0', grad_fn=<NllLossBackward0>)
382000
38600 tensor(5.3713, device='cuda:0', grad_fn=<NllLossBackward0>)
383000
38700 tensor(5.3634, device='cuda:0', grad_fn=<NllLossBackward0>)
384000
38800 tensor(5.4504, device='cuda:0', grad_fn=<NllLossBackward0>)
385000
38900 tensor(5.5273, device='cuda:0', grad_f

52000
5300 tensor(5.5077, device='cuda:0', grad_fn=<NllLossBackward0>)
53000
5400 tensor(5.3746, device='cuda:0', grad_fn=<NllLossBackward0>)
54000
5500 tensor(5.4847, device='cuda:0', grad_fn=<NllLossBackward0>)
55000
5600 tensor(5.2664, device='cuda:0', grad_fn=<NllLossBackward0>)
56000
5700 tensor(5.5265, device='cuda:0', grad_fn=<NllLossBackward0>)
57000
5800 tensor(5.5101, device='cuda:0', grad_fn=<NllLossBackward0>)
58000
5900 tensor(5.4513, device='cuda:0', grad_fn=<NllLossBackward0>)
59000
6000 tensor(5.3554, device='cuda:0', grad_fn=<NllLossBackward0>)
60000
6100 tensor(5.3616, device='cuda:0', grad_fn=<NllLossBackward0>)
61000
6200 tensor(5.5360, device='cuda:0', grad_fn=<NllLossBackward0>)
62000
6300 tensor(5.3952, device='cuda:0', grad_fn=<NllLossBackward0>)
63000
6400 tensor(5.3132, device='cuda:0', grad_fn=<NllLossBackward0>)
64000
6500 tensor(5.1732, device='cuda:0', grad_fn=<NllLossBackward0>)
65000
6600 tensor(5.3505, device='cuda:0', grad_fn=<NllLossBackward0>)
66000


165000
16700 tensor(5.3130, device='cuda:0', grad_fn=<NllLossBackward0>)
166000
16800 tensor(5.3903, device='cuda:0', grad_fn=<NllLossBackward0>)
167000
16900 tensor(5.1551, device='cuda:0', grad_fn=<NllLossBackward0>)
168000
17000 tensor(5.4229, device='cuda:0', grad_fn=<NllLossBackward0>)
169000
17100 tensor(5.2686, device='cuda:0', grad_fn=<NllLossBackward0>)
170000
17200 tensor(5.4099, device='cuda:0', grad_fn=<NllLossBackward0>)
171000
17300 tensor(5.4198, device='cuda:0', grad_fn=<NllLossBackward0>)
172000
17400 tensor(5.2162, device='cuda:0', grad_fn=<NllLossBackward0>)
173000
17500 tensor(5.2385, device='cuda:0', grad_fn=<NllLossBackward0>)
174000
17600 tensor(5.2786, device='cuda:0', grad_fn=<NllLossBackward0>)
175000
17700 tensor(5.2576, device='cuda:0', grad_fn=<NllLossBackward0>)
176000
17800 tensor(5.3158, device='cuda:0', grad_fn=<NllLossBackward0>)
177000
17900 tensor(5.2105, device='cuda:0', grad_fn=<NllLossBackward0>)
178000
18000 tensor(5.4627, device='cuda:0', grad_f

277000
28000 tensor(5.3454, device='cuda:0', grad_fn=<NllLossBackward0>)
278000
28100 tensor(5.3113, device='cuda:0', grad_fn=<NllLossBackward0>)
279000
28200 tensor(5.4785, device='cuda:0', grad_fn=<NllLossBackward0>)
280000
28300 tensor(5.1013, device='cuda:0', grad_fn=<NllLossBackward0>)
281000
28400 tensor(5.5403, device='cuda:0', grad_fn=<NllLossBackward0>)
282000
28500 tensor(5.3676, device='cuda:0', grad_fn=<NllLossBackward0>)
283000
28600 tensor(5.3108, device='cuda:0', grad_fn=<NllLossBackward0>)
284000
28700 tensor(5.4403, device='cuda:0', grad_fn=<NllLossBackward0>)
285000
28800 tensor(5.4926, device='cuda:0', grad_fn=<NllLossBackward0>)
286000
28900 tensor(5.3638, device='cuda:0', grad_fn=<NllLossBackward0>)
287000
29000 tensor(5.2819, device='cuda:0', grad_fn=<NllLossBackward0>)
288000
29100 tensor(5.0362, device='cuda:0', grad_fn=<NllLossBackward0>)
289000
29200 tensor(5.1871, device='cuda:0', grad_fn=<NllLossBackward0>)
290000
29300 tensor(5.4697, device='cuda:0', grad_f

389000
39300 tensor(5.3452, device='cuda:0', grad_fn=<NllLossBackward0>)
390000
39400 tensor(5.4111, device='cuda:0', grad_fn=<NllLossBackward0>)
391000
39500 tensor(5.4969, device='cuda:0', grad_fn=<NllLossBackward0>)
392000
39600 tensor(5.1952, device='cuda:0', grad_fn=<NllLossBackward0>)
393000
39700 tensor(5.1946, device='cuda:0', grad_fn=<NllLossBackward0>)
394000
39800 tensor(5.3234, device='cuda:0', grad_fn=<NllLossBackward0>)
395000
39900 tensor(5.1354, device='cuda:0', grad_fn=<NllLossBackward0>)
396000
40000 tensor(5.2210, device='cuda:0', grad_fn=<NllLossBackward0>)
397000
40100 tensor(5.3133, device='cuda:0', grad_fn=<NllLossBackward0>)
398000
40200 tensor(5.2990, device='cuda:0', grad_fn=<NllLossBackward0>)
399000
40300 tensor(5.3684, device='cuda:0', grad_fn=<NllLossBackward0>)
40400 tensor(5.3700, device='cuda:0', grad_fn=<NllLossBackward0>)
400000
40500 tensor(5.2911, device='cuda:0', grad_fn=<NllLossBackward0>)
401000
40600 tensor(5.3497, device='cuda:0', grad_fn=<NllL

In [17]:
device = 'cuda'
torch.cuda.empty_cache()
model = SimpleBigramNeuralLanguageModel(vocab_size, embed_size).to(device)
model.load_state_dict(torch.load(f'model-bigram_final.bin'))
model.eval()

ixs = torch.tensor(vocab.forward(['will'])).to(device)

out = model(ixs)
top = torch.topk(out[0], 10)
top_indices = top.indices.tolist()
top_probs = top.values.tolist()
top_words = vocab.lookup_tokens(top_indices)
list(zip(top_words, top_indices, top_probs))

[('be', 11, 0.2570849657058716),
 ('<unk>', 0, 0.07411641627550125),
 ('not', 22, 0.05940083786845207),
 ('have', 28, 0.02751326560974121),
 ('bo', 167, 0.014936885796487331),
 ('make', 116, 0.013943656347692013),
 ('give', 193, 0.011286991648375988),
 ('take', 153, 0.011171611957252026),
 ('do', 86, 0.010088067501783371),
 ('he', 20, 0.009703895077109337)]

In [34]:
vocab = train_dataset.vocab
ixs = torch.tensor(vocab.forward(['cerned.'])).to(device)

out = model(ixs)
top = torch.topk(out[0], 10)
top_indices = top.indices.tolist()
top_probs = top.values.tolist()
top_words = vocab.lookup_tokens(top_indices)
list(zip(top_words, top_indices, top_probs))

[('<unk>', 0, 0.19996878504753113),
 ('and', 3, 0.05288130044937134),
 ('of', 2, 0.042051784694194794),
 ('the', 1, 0.026572922244668007),
 ('to', 4, 0.022689413279294968),
 ('in', 6, 0.015904497355222702),
 ('The', 17, 0.012827681377530098),
 ('a', 5, 0.00961760152131319),
 ('for', 8, 0.008938422426581383),
 ('</s>', 32, 0.00840282253921032)]

In [33]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

embeddings = model.model[0].weight

vec = embeddings[vocab['cerned.']]

similarities = cos(vec, embeddings)

top = torch.topk(similarities, 10)

top_indices = top.indices.tolist()
top_probs = top.values.tolist()
top_words = vocab.lookup_tokens(top_indices)
list(zip(top_words, top_indices, top_probs))

[('<unk>', 0, 1.0),
 ('particular,', 14538, 0.24527804553508759),
 ('revolution.', 20446, 0.23776617646217346),
 ('Territory.', 14189, 0.23417341709136963),
 ('or-', 2261, 0.22888363897800446),
 ('3', 479, 0.2288265973329544),
 ('speak.', 13722, 0.2252315878868103),
 ('attend.', 19397, 0.22110989689826965),
 ('say,', 1455, 0.22106117010116577),
 ('Lee.', 15326, 0.21764159202575684)]

In [ ]:

vocab = train_dataset.vocab
# ixs = torch.tensor(vocab.forward(['a'])).to(device)
ixs = torch.tensor(vocab.forward(['of'])).to(device)
# ixs = torch.tensor(vocab.forward(['that'])).to(device)
# ixs = torch.tensor(vocab.forward(['church'])).to(device)
# ixs = torch.tensor(vocab.forward(['wait'])).to(device)

out = model(ixs)
top = torch.topk(out[0], 10)
top_indices = top.indices.tolist()
top_probs = top.values.tolist()
top_words = vocab.lookup_tokens(top_indices)
list(zip(top_words, top_indices, top_probs))

In [52]:
def get_values_from_model(presc_word, model, vocab, k):
    ixs =  torch.tensor(vocab.forward([presc_word])).to(device)
    out = model(ixs)
    top = torch.topk(out[0], k)
    top_indices = top.indices.tolist()
    top_probs = top.values.tolist()
    top_words = vocab.lookup_tokens(top_indices)
    return list(zip(top_words, top_probs))

def gonito_format(dic):
    tab = summarize_probs_unk(dic)
    result = ''
    for element in tab[:-1]:
        result+=str(element[0])+':'+str(element[1])+'\t'
    result+=':'+ str(tab[-1][1])+'\n'
    return result

def summarize_probs_unk(dic):
    if '<unk>' in dic.keys():
        del dic['<unk>']
    probsum = sum(float(val) for key, val in dic.items())
#     if "<unk>" in dic.keys():
#         for key in dic:
#             dic[key] = dic[key]/probsum #normalize to 1
#         wildcard = dic['<unk>'] 
#     else: #no unk in first 10 entries
    for key in dic:
        dic[key] = dic[key]/probsum*(1-wildcard_minweight) ###leave some space for wildcard
    wildcard = wildcard_minweight
    tab = [(key, val) for key, val in dic.items()]
    tab.append(('<unk>', wildcard))
    return tab



In [54]:

with lzma.open(test_file, 'rt') as file:
    predict_words = []
    results = []
    for line in file:
#         print(line)
        line = preprocess(line) #get only relevant
        split = line.split('\t')
        print(get_last_word(split[0]))
        predict_words.append(get_last_word(split[0])) #get_first_word(split[1])
    print(len(predict_words))
    vocab = train_dataset.vocab
    for presc_word in predict_words:
        results.append(dict(get_values_from_model(presc_word, model, vocab, k=k)))
    
    
    with open(out_file, 'w') as outfile:
        for elem in results:
            
            outfile.write(gonito_format(elem))



Present,
maintenance.
-
on
after.
included.
Delaware.
body.
route.
under
bership.
call*
waterscape.
?
soles.
simple."
public.
them.-
months.
icy."
E
level.
4t—36
financial
steady.
mlnicars.
his
circumstances.
corner.
science.
insurance."
fair.
scalded.
ria.
it,
mlngtonlan.
possession.
of
pray!”
occurred.
euro,
way.
185.
destruction.
conditions.
Hood.
s.
case.
Roy
country.
ccjulinalf
side.
against
servation.
Detroit.
North
country.
"
bones.
sold.
Brown
bill.
Springer,
gaged.
their
character.
cu
former
cities.
accumulated
corporal.
^
-
o
.<!.
them.
wounded.
by.
of
remedy.
debate.
22-28).
"
book."
contract,
presentation.
Ellsha
praise.
opin-
proof.
lows:
come
dispensation.
point.
theirjfnmiliea.
ull.
Magazine.
rnUnil.
order.
morrow
nrL
is
'death.
caucus.
east
-
service.
cor-
ton
answer.
bay.
same.
:
Arkansas.
lives.
the
t
.1
P
lems?
4,
know,'
his
arouth."
offer.
horizontal!'.
two-thir-
city.
come
here.
STORY
auxiliaries.
property.
respected.
stopped.
JanlO
honesty.
civilian
servation.
arm

Burgess)
cheap.
ENOUGH
,
lor
by
over.
JAA41»
tory
belonging,
on
style:
L'XJK'IulitUrfH.
bill!
subscription
in-
crats.
them.
tho
velt
furnished.
acts.
ers.
wasted
Nevada,
harbor.
surrendered
see
knives.
evei
world:
.
Promotions.
"
ways.
buried
extra.
J.
promote.
nay.
dr.
less.
'coop.'
Country.
wife.
rot."
calculated.
education.
person,
»
children.
ILLY,
Bear.
ways.
!«-
and
management.
ants.
commodity.
scenes
tribune.
wide-sprea-
costs.
success."
ter
indopen-deuce-
a
La'jle.
after
pose.
lauding.
llowers,
Hoursfrom9a.m..todr.m.
year.
Keswick's
forever
us,
witness
Spetoh
living-situation.
e
.
other.
helpful.
another.
exhib.:
best
so
animated.
says
arrests.
abiding
America.
¬
t
says:
efface.
hereafter.
fit.”
Iwentv-tdne
eorners
I.
the
night.
immediately.
law.
club.'
editors.
McDonough
Stomp,"
?
as
erty."
evening,
manafar-
C.
distress.
(SKAL.I
States.
without.
e
school.
dealers.
gs
thing.
names.
fo
opposed.
parent
greatly
ber
lce.
their
privilege.
Cost*.
Wlikes-Barr-
Sentinel.
13.
Patriot.
n